In [66]:
import geopandas as gpd
import pandas as pd
import rasterio as rio
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata


In [67]:
# import os
g = os.getcwd()
g
path = r'D:\repos\GEO_CLARO'
os.chdir(path)

In [6]:
pnt = gpd.read_file('data/points_ok/late_bgt.shp')
pnt['FECHA'] = pd.to_datetime(pnt['FECHA'])
pnt["N_OPERADOR"] = pnt["N_OPERADOR"].astype(int)
pnt.head(10)


,FECHA,TEST_ID,N_OPERADOR,DOWNLOAD_M,UPLOAD_MBP,CLASS_LATE,localidad,geometry
0,2021-01-28,6978771403,4,10.410156,0.640625,0,ANTONIO NARIÑO,POINT (-74.09600 4.58700)
1,2021-01-19,6945885369,4,10.294922,4.988281,0,ANTONIO NARIÑO,POINT (-74.12100 4.59200)
2,2021-01-08,6907797685,4,28.707031,4.711914,0,ANTONIO NARIÑO,POINT (-74.10000 4.59000)
3,2021-01-21,6953823661,4,53.502930,7.201172,0,ANTONIO NARIÑO,POINT (-74.09500 4.58500)
4,2021-01-21,6952553476,4,26.608398,6.316406,0,ANTONIO NARIÑO,POINT (-74.12100 4.59300)
5,2021-01-21,6954103035,4,5.515625,4.158203,0,ANTONIO NARIÑO,POINT (-74.10300 4.59000)
6,2021-01-23,6961155289,4,48.430664,6.875977,0,ANTONIO NARIÑO,POINT (-74.10400 4.59100)
7,2021-01-23,6960257415,4,3.519531,2.939453,0,ANTONIO NARIÑO,POINT (-74.10600 4.59200)
8,2021-01-25,6966330006,4,0.201172,1.002930,0,ANTONIO NARIÑO,POINT (-74.09400 4.58600)
9,2021-01-25,6967943641,4,7.100586,4.663086,0,ANTONIO NARIÑO,POINT (-74.08900 4.58900)


In [7]:
pnt.drop(columns=['TEST_ID','DOWNLOAD_M','UPLOAD_MBP','CLASS_LATE','localidad'], inplace=True)
pnt.head(10)

,FECHA,N_OPERADOR,geometry
0,2021-01-28,4,POINT (-74.09600 4.58700)
1,2021-01-19,4,POINT (-74.12100 4.59200)
2,2021-01-08,4,POINT (-74.10000 4.59000)
3,2021-01-21,4,POINT (-74.09500 4.58500)
4,2021-01-21,4,POINT (-74.12100 4.59300)
5,2021-01-21,4,POINT (-74.10300 4.59000)
6,2021-01-23,4,POINT (-74.10400 4.59100)
7,2021-01-23,4,POINT (-74.10600 4.59200)
8,2021-01-25,4,POINT (-74.09400 4.58600)
9,2021-01-25,4,POINT (-74.08900 4.58900)


In [8]:
pnt[['N_OPERADOR','FECHA']].describe(include='all')

C:\Users\FRANKH~1\AppData\Local\Temp/ipykernel_11192/1654886183.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pnt[['N_OPERADOR','FECHA']].describe(include='all')


,N_OPERADOR,FECHA
count,4.747929e+06,4747929
unique,NaN,270
top,NaN,2021-04-29 00:00:00
freq,NaN,42960
first,NaN,2021-01-01 00:00:00
last,NaN,2021-09-27 00:00:00
mean,2.809081e+00,NaN
std,1.348064e+00,NaN
min,1.000000e+00,NaN
25%,1.000000e+00,NaN


In [9]:
pnt_copy = pnt.copy()
pnt_copy = pnt_copy[pnt_copy['FECHA']>='2021-04-01']
pnt_copy[['N_OPERADOR','FECHA']].describe(include='all')


C:\Users\FRANKH~1\AppData\Local\Temp/ipykernel_11192/3229697023.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pnt_copy[['N_OPERADOR','FECHA']].describe(include='all')


,N_OPERADOR,FECHA
count,2.997973e+06,2997973
unique,NaN,180
top,NaN,2021-04-29 00:00:00
freq,NaN,42960
first,NaN,2021-04-01 00:00:00
last,NaN,2021-09-27 00:00:00
mean,2.771873e+00,NaN
std,1.349178e+00,NaN
min,1.000000e+00,NaN
25%,1.000000e+00,NaN


In [11]:
pnt_copy = pnt_copy.to_crs("EPSG:3116")

In [68]:
def rasterize_gpd (ponist, column, path_out, resM=[30,30]):
    '''
    ponist= geopandas puntos
    column= columna a rasteizar, 
    path_out= dir de salida formato tif , 
    resM= resolucion en metrod

    '''
    cube = make_geocube(
        ponist.to_crs("EPSG:3116"),
        measurements=[column],
        resolution=(resM[0], resM[1]),
        fill=0,
        # rasterize_function=rasterize_points_griddata,
    )
    cube[column].rio.to_raster(path_out)
    return print('Done')

In [69]:
for date in pnt_copy['FECHA'].unique():
    d = pnt_copy[pnt_copy.FECHA==date]
    for operador in pnt_copy.N_OPERADOR.unique():
        o = d[d.N_OPERADOR==operador].reset_index()
        print('voy en esta fecha: ',str(date), ' con este operador: ',str(operador))
        rasterize_gpd(o,'N_OPERADOR', 'data/img/'+str(operador)+'/'+str(date).split('T')[0]+'.tif', [100,100])


voy en esta fecha:  2021-04-01T00:00:00.000000000  con este operador:  1
Done
voy en esta fecha:  2021-04-01T00:00:00.000000000  con este operador:  3
Done
voy en esta fecha:  2021-04-01T00:00:00.000000000  con este operador:  2
Done
voy en esta fecha:  2021-04-01T00:00:00.000000000  con este operador:  4
Done
voy en esta fecha:  2021-04-07T00:00:00.000000000  con este operador:  1
Done
voy en esta fecha:  2021-04-07T00:00:00.000000000  con este operador:  3
Done
voy en esta fecha:  2021-04-07T00:00:00.000000000  con este operador:  2
Done
voy en esta fecha:  2021-04-07T00:00:00.000000000  con este operador:  4
Done
voy en esta fecha:  2021-04-13T00:00:00.000000000  con este operador:  1
Done
voy en esta fecha:  2021-04-13T00:00:00.000000000  con este operador:  3
Done
voy en esta fecha:  2021-04-13T00:00:00.000000000  con este operador:  2
Done
voy en esta fecha:  2021-04-13T00:00:00.000000000  con este operador:  4
Done
voy en esta fecha:  2021-04-15T00:00:00.000000000  con este oper

In [17]:
1.2*4*180

2649.6

In [65]:
pnt_copy.N_OPERADOR.unique()

array([1, 3, 2, 4])